# Retriever and Chain with Langchain

In [1]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("Pokemon-ultra-violet-guide.pdf")
docs = loader.load()
docs

[Document(page_content='      Welcome to the Official Players Guide for Pokemon - Ultra Violet Version.  \nFor those of you who do not know, Pokemon – Ultra Violet Version  is a hack \nof Pokemon – Fire Red Version . The story and gameplay will be the same but \nthere will be a lot of added content. There are a t otal of 386 Pokemon in this \ngame, all are actually catchable in this ONE game, no trading is necessary.  \n     This guide will contain notes from the hacker who created the ROM and \nwill give you all the inside tips on where to catch  every single Pokemon and \nall the other fun additions to the original game. \nSo without further ado, let’s catch ‘em all!  \nUsing This Guide  \nThis guide has three major parts: \nGame Intro  \n      The first section covers all the basics and i mportant \nstuff you need to know before you start the game.  The Index  \n     Filled with tables, the Index is where you go to find \nlocations on everything in the game, from items to  \nThe Wal

In [74]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
text_splitter.split_documents(docs)[:5]

[Document(page_content='Welcome to the Official Players Guide for Pokemon - Ultra Violet Version.  \nFor those of you who do not know, Pokemon – Ultra Violet Version  is a hack \nof Pokemon – Fire Red Version . The story and gameplay will be the same but \nthere will be a lot of added content. There are a t otal of 386 Pokemon in this \ngame, all are actually catchable in this ONE game, no trading is necessary.  \n     This guide will contain notes from the hacker who created the ROM and', metadata={'source': 'Pokemon-ultra-violet-guide.pdf', 'page': 0}),
 Document(page_content='This guide will contain notes from the hacker who created the ROM and \nwill give you all the inside tips on where to catch  every single Pokemon and \nall the other fun additions to the original game. \nSo without further ado, let’s catch ‘em all!  \nUsing This Guide  \nThis guide has three major parts: \nGame Intro  \n      The first section covers all the basics and i mportant \nstuff you need to know before

In [75]:
documents=text_splitter.split_documents(docs)
documents

[Document(page_content='Welcome to the Official Players Guide for Pokemon - Ultra Violet Version.  \nFor those of you who do not know, Pokemon – Ultra Violet Version  is a hack \nof Pokemon – Fire Red Version . The story and gameplay will be the same but \nthere will be a lot of added content. There are a t otal of 386 Pokemon in this \ngame, all are actually catchable in this ONE game, no trading is necessary.  \n     This guide will contain notes from the hacker who created the ROM and', metadata={'source': 'Pokemon-ultra-violet-guide.pdf', 'page': 0}),
 Document(page_content='This guide will contain notes from the hacker who created the ROM and \nwill give you all the inside tips on where to catch  every single Pokemon and \nall the other fun additions to the original game. \nSo without further ado, let’s catch ‘em all!  \nUsing This Guide  \nThis guide has three major parts: \nGame Intro  \n      The first section covers all the basics and i mportant \nstuff you need to know before

In [76]:
# from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents[:30], OllamaEmbeddings(model="llama3"))

In [77]:
from langchain_community.llms import Ollama
## Load gemma LLM model
llm=Ollama(model="llama3")
llm

Ollama(model='llama3')

In [91]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Imagine you are an expert in Pokemon. 
Your role is to help the user by answering their question about the Pokemon game.
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer.
Answer in a natural conversational way. 
Give answers that the user finds helpful.
<context>
{context}
</context>
Question: {input}""")

In [92]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [93]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002254EC59600>)

In [94]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [95]:
response=retrieval_chain.invoke({"input":"What is the Pokemon Ultra Violet Version game about?"})
print(response['answer'])

A new Pokémon trainer, eh? Well, let me tell you all about Pokémon Ultra Violet Version!

From what I gathered from the context, Pokémon Ultra Violet Version is a special edition of the classic Pokémon games. It seems that trading with other players isn't as crucial in this version, unlike some other games where certain Pokémon require trades to evolve.

What's unique about this game is that you can catch and evolve every single Pokémon without needing to trade for any from another game! That's right; no more worrying about finding a specific Pokémon in someone else's collection. You're free to explore and catch 'em all on your own!

As an expert, I'd say that Pokémon Ultra Violet Version is perfect for those who love the thrill of adventure, exploring new places, and building relationships with their trusty Pokémon companions. It seems like this game offers a unique twist on the traditional Pokémon formula, giving you more control over your journey.

So, what are you waiting for? Grab

In [96]:
response=retrieval_chain.invoke({"input":"How many pokemon types are there?"})
print(response['answer'])

A great question for a Pokémon expert like myself!

According to the context, there isn't any specific mention of Pokémon types. However, based on my expertise and knowledge of the Pokémon series, I can tell you that there are 18 recognized Pokémon types in the franchise.

These types include Normal, Fire, Water, Grass, Electric, Ice, Fighting, Poison, Ground, Flying, Psychic, Bug, Rock, Ghost, Steel, Dragon, Dark, and Fairy.

Keep in mind that this information is not explicitly stated in the provided context. It's based on my general knowledge of the Pokémon series, which I'm happy to share with you!
